# HMDA Cleaner

The purpose of this program is to clean HMDA data further data needs.

This program is designed to be used for future data HMDA publications.

## Import HMDA Data, Packages, and Check Dataset.

In [3]:
#This code does not need to be changed.
#Imports packages to be used by program
import pandas as pd
import numpy as np

In [4]:
#This code does not need to be changed.
#Quality of Life settings
#This will allow all columns to be displayed when reviewing the data.
pd.options.display.max_columns = None

In [5]:
#Loads in Sample
HMDA_dataset_file_location = r'2019_HMDA_sample.csv'

In [6]:
#Sets Sample as dataframe.
HMDA_dataset = pd.read_csv(HMDA_dataset_file_location)
HMDA_dataset

activity_year                   lei  derived_msa-md state_code  \
0            2019  5493000YNV8IX4VD3X12           99999         KY   
1            2019  549300O6Z0I6KYMESL47           16984         IL   
2            2019  549300XRXBA38J60S618           99999         ID   
3            2019  549300NBFJM2PBH24L77           48620         KS   
4            2019  C5654JQHZUHN0772B561           47894         VA   
..            ...                   ...             ...        ...   
95           2019  254900FL0I8OUMMBOZ11           38060         AZ   
96           2019  549300NOCASXPA34X033           15180         TX   
97           2019  B4TYDEB6GKMZO031MB27           22744         FL   
98           2019  549300CY7WNAHKHYSJ73           99999         WA   
99           2019  549300YOESI1GLKRL151           16740         SC   

    county_code  census_tract conforming_loan_limit  \
0       21187.0  2.118797e+10                     C   
1       17031.0  1.703181e+10                     C   
2       16039.0  1.603996e+10                     C   
3       20173.0  2.017301e+10                     C   
4       51683.0  5.168391e+10                     C   
..          ...           ...                   ...   
95       4013.0  4.013072e+09                     C   
96      48061.0  4.806101e+10                     C   
97      12011.0  1.201104e+10                     C   
98      53037.0  5.303798e+10                    NC   
99      45091.0  4.509106e+10                     C   

        derived_loan_product_type               derived_dwelling_category  \
0         Conventional:First Lien  Single Family (1-4 Units):Manufactured   
1         Conventional:First Lien    Single Family (1-4 Units):Site-Built   
2         Conventional:First Lien    Single Family (1-4 Units):Site-Built   
3         Conventional:First Lien  Single Family (1-4 Units):Manufactured   
4                   VA:First Lien    Single Family (1-4 Units):Site-Built   
..                            ...                                     ...   
95        Conventional:First Lien    Single Family (1-4 Units):Site-Built   
96        Conventional:First Lien    Single Family (1-4 Units):Site-Built   
97  Conventional:Subordinate Lien    Single Family (1-4 Units):Site-Built   
98        Conventional:First Lien    Single Family (1-4 Units):Site-Built   
99                 FHA:First Lien    Single Family (1-4 Units):Site-Built   

          derived_ethnicity                               derived_race  \
0    Not Hispanic or Latino                                      White   
1        Hispanic or Latino                                      White   
2    Not Hispanic or Latino                                      White   
3    Not Hispanic or Latino                                      White   
4    Not Hispanic or Latino                                      White   
..                      ...                                        ...   
95  Ethnicity Not Available  Native Hawaiian or Other Pacific Islander   
96  Ethnicity Not Available                                      White   
97  Ethnicity Not Available                         Race Not Available   
98   Not Hispanic or Latino                                      White   
99  Ethnicity Not Available                         Race Not Available   

          derived_sex  action_taken  purchaser_type  preapproval  loan_type  \
0              Female             5               0            2          1   
1               Joint             4               0            2          1   
2                Male             1               0            2          1   
3                Male             1               0            2          1   
4                Male             1               2            2          3   
..                ...           ...             ...          ...        ...   
95               Male             1               0            2          1   
96               Male             3               0     

In [110]:
# HMDA_dataset.columns

In [113]:
'''
This code does not need to be changed unless column headers change.(Ex.'state_code' changes to 'state')
Program to call for future HDMA data.
'''
def HMDA_cleaner(HMDA_dataset_file_location):
    #load in dataset.
    HMDA_dataset = pd.read_csv(HMDA_dataset_file_location)
    
    '''
    Clean Race data.
    '''
    #Groups racial categories and adds names.
    df1 = HMDA_dataset.replace(to_replace = {'applicant_race-1' : {
                                (2,21,22,23,24,25,26,27):'Asian',
                                  (3):'Black',
                                 (5):'White',
                                 (1, 4,41,42,43,44):'Other',
                                 (6,7, np.nan):'Not Reported'}})
    #Remove 'Not Reported' rows.
    df2 = df1[df1['applicant_race-1'] != 'Not Reported']
    #Clean Ethnicity Data.
    df3 = df2.replace(to_replace = {'applicant_ethnicity-1' : {
                                (1,11,14,12,13):'Hispanic',
                                  (2):'Not Hispanic',
                                 (3,4,np.nan):'Not Reported',}})
    #Remove 'Not Reported' rows.
    df4 = df3[df3['applicant_ethnicity-1'] != 'Not Reported']
    #Get dummy variables for applicant race and ethnicity.
    df5 = pd.get_dummies(df4, columns = ['applicant_race-1', 'applicant_ethnicity-1'])
    
    '''
    Clean Income 
    '''
    #Remove rows that had no income reported or an income less than 0.
    def income_reported_to_categorical(table, column):
        values = table[column].values
        ir_array =  []
        for i in values:
            if i == np.nan:
                ir_array.append('No Report')
            elif i <= 0:
                ir_array.append('Zero or Negative')
            elif i > 0:
                ir_array.append('Reported')
            else:
                ir_array.append('No Report')
        table['Income Reported'] = ir_array     
        return table
    #This removes all non reported and 0 or negative rows.
    df6 = income_reported_to_categorical(df5,'income')
    df7 = df6[df6["Income Reported"].str.contains("Reported") == True]
    #Creates log Income column.
    df7_2 = df7.copy()
    df7_2['Log Income'] = np.log(df7['income'])
    
    '''
    Create approval and denial indicator column.
    This also removes withdrawn applications and other instances that aren't under the Approval or Denial categories.
    Preapproval request indicators could also be used as an indicator variable if need be.
    '''
    def action_taken_to_approval_indicator(table, column):
        values = table[column].values
        ir_array =  []
        for i in values:
            if i == 1:
                ir_array.append('Approved')
            elif i == 2:
                ir_array.append('Approved')
            elif i == 3:
                ir_array.append('Denied')
            else:
                ir_array.append('other')
        table2 = table.copy()
        table2['Approval Indicator'] = ir_array
        table3 = table2[table2["Approval Indicator"].str.contains("other") == False]
        table4 = pd.get_dummies(table3, columns = ['Approval Indicator'])
        return table4
    df8 = action_taken_to_approval_indicator(df7_2, 'action_taken')
   
    '''
    Clean applicant gender
    '''
    df9 = df8.replace(to_replace = {'applicant_sex' : {(1):'Male',(2):'Female'}})
    df10 = pd.get_dummies(df9, columns = ['applicant_sex'])
    
    '''
    Clean Loan to Value Ratios
    '''
    #This removes all non-numeric rows.
    df11 = df10[pd.to_numeric(df10['loan_to_value_ratio'], errors = 'coerce').notnull()]
    
    '''
    Clean Debt to Income    
    '''
    #Drops no answer rows.
    df12 = df11.dropna(subset=['debt_to_income_ratio'])
    #Drops exempt rows.
    df13 = df12[df12["debt_to_income_ratio"].str.contains("Exempt") == False]
    #Rename Column
    df13_5 = df13.rename(columns = {'debt_to_income_ratio': 'DTI'})
    df13_4 =  pd.get_dummies(df13_5, columns = ['DTI'])
   
    
    '''
    Clean Loan Amount and create log(Loan Amount)
    '''
    #This drops all N/A rows.
    df14 = df13_4.dropna(subset=['loan_amount'])
    #This removes all non-numeric rows.
    df15 = df14[pd.to_numeric(df14['loan_amount'], errors = 'coerce').notnull()]
    #This removes all rows with a value less than or equal to 0.
    df16 = df15[~(df15['loan_amount'] <= 0)]
    #Creates log(Loan Amount)
    df16['Log Loan Amount'] = np.log(df16['loan_amount'])
    
    '''
    Clean Loan Type
    '''
    df17 = df16.replace(to_replace = {'loan_type' : {(1):'Conventional',
                                                    (2): 'FHA',(3): 'VA',
                                                    #USDA Rural Housing Service or Farm Service Agency.
                                                    (4): 'RHS or FSA'}})
    df18 = df17.rename(columns = {'loan_type': 'Loan Type'})
    df19 = pd.get_dummies(df18, columns = ['Loan Type'])
    
    '''
    Filter Loan Purpose
    '''
    #This study is only interested in Home Purchase loans.
    #It is leaving out home improvement, refinancing, Cash-out refinanciing, and other loan purposes.
    df20 = df19[df12["loan_purpose"] == 1]
    
    '''
    Compile final Data Frame
    '''
    #Rename columns
    dfrename = df20.rename(columns = {'Approval Indicator_Approved' : 'Approved',
                                       'Approval Indicator_Denied' : 'Denied',
                                        'income': 'Income',
                                        'applicant_race-1_Asian': 'Asian',
                                        'applicant_race-1_Black': 'Black',
                                        'applicant_race-1_Other': 'Other',
                                        'applicant_race-1_White': 'White',
                                        'applicant_ethnicity-1_Hispanic': 'Hispanic',
                                        'applicant_ethnicity-1_Not Hispanic': 'Not Hispanic',
                                        'applicant_sex_Female': 'Female',
                                        'applicant_sex_Male': 'Male',
                                        'loan_to_value_ratio': 'LTV',
                                        'loan_amount': 'Loan Amount',
                                        'denial_reason-1': 'Denial Reason',
                                        'state_code' : 'State',
                                        'county_code':'County Code',
                                        'census_tract': 'Census Tract'})
    #Pull columns
    Final_df = dfrename[['State','County Code','Census Tract', 'Approved', 'Denied', 
                         'White', 'Black', 'Asian', 'Other', 'Hispanic', 'Not Hispanic', 
                         'Male', 'Female','Income', 'Log Income', 
                         'Loan Amount', 'Log Loan Amount','LTV','Loan Type_Conventional', 
                         'Loan Type_FHA','Loan Type_VA','Loan Type_RHS or FSA','DTI_20%-<30%',
                         'DTI_30%-<36%','DTI_36','DTI_38','DTI_39','DTI_40','DTI_41',
                         'DTI_42','DTI_43','DTI_44','DTI_45','DTI_46','DTI_47',
                         'DTI_48','DTI_50%-60%','DTI_<20%','DTI_>60%']]
    
    '''
    Save the Dataframe to a csv
    '''
    Final_df.to_csv(r'2019 HMDA Clean.csv')
    
    '''
    Outputs
    '''
    print('Data written to CSV sucessfully.')
    print(Final_df.shape[0],'Rows,',Final_df.shape[1],'Columns.')
    return Final_df.head()

In [114]:
#Run HMDA_cleaner.
#Enter file location in the HMDA_cleaner(r'file_location') format.
HMDA_cleaner(r'2019_HMDA_sample.csv')

Data written to CSV sucessfully.
27 Rows, 39 Columns.


State  County Code  Census Tract  Approved  Denied  White  Black  Asian  \
4     VA      51683.0  5.168391e+10         1       0      1      0      0   
8     WI      55133.0  5.513320e+10         1       0      1      0      0   
9     FL      12101.0  1.210103e+10         0       1      0      1      0   
10    TX      48157.0  4.815767e+10         0       1      0      1      0   
12    TX      48121.0  4.812102e+10         1       0      1      0      0   

    Other  Hispanic  Not Hispanic  Male  Female  Income  Log Income  \
4       0         0             1     1       0   115.0    4.744932   
8       0         0             1     0       1    71.0    4.262680   
9       0         0             1     0       1    42.0    3.737670   
10      0         0             1     0       1   203.0    5.313206   
12      0         0             1     1       0    75.0    4.317488   

    Loan Amount  Log Loan Amount    LTV  Loan Type_Conventional  \
4      395000.0        12.886641  100.0                       0   
8      225000.0        12.323856  95.79                       1   
9      185000.0        12.128111   80.0                       1   
10      85000.0        11.350407   90.0                       1   
12     295000.0        12.594731  100.0                       0   

    Loan Type_FHA  Loan Type_VA  Loan Type_RHS or FSA  DTI_20%-<30%  \
4               0             1                     0             0   
8               0             0                     0             1   
9               0             0                     0             0   
10              0             0                     0             0   
12              0             1                     0             0   

    DTI_30%-<36%  DTI_36  DTI_38  DTI_39  DTI_40  DTI_41  DTI_42  DTI_43  \
4              1       0       0       0       0       0       0       0   
8              0       0       0       0       0       0       0       0   
9              0       0       0       0       0       0       0       0   
10             0       0       0       0       0       0       1       0   
12             0       0       0       0       1       0       0       0   

    DTI_44  DTI_45  DTI_46  DTI_47  DTI_48  DTI_50%-60%  DTI_<20%  DTI_>60%  
4        0       0       0       0       0            0         0         0  
8        0       0       0       0       0            0         0         0  
9        0       0       0       0       0            1         0         0  
10       0       0       0       0       0            0         0         0  
12       0       0       0       0       0            0         0         0

### The CSV is now ready to be used for further descriptive statistics and the LPM.